In [5]:
import numpy as np

import qiskit
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, assemble
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import PulseSimulator, Aer, QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.pulse import PulseSystemModel
from qiskit.test.mock import FakeArmonk
from qiskit.tools import job_monitor
from helper import qutip_ham_converter
from qoc_instruction_schedule_map import QOCInstructionScheduleMap
from QutipOptimizer import QutipOptimizer

IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
real_armonk_backend = provider.get_backend('ibmq_armonk')

ImportError: attempted relative import with no known parent package

In [ ]:
fake_armonk_backend=FakeArmonk()
# which_armonk = 'fake'
which_armonk = 'fake'
def_x = False
if which_armonk=='real':
    fake_armonk_backend=real_armonk_backend
# doesn't work with name()'

In [ ]:
# convert hamiltonian from backend to qutip nbformat
hamiltonian = qutip_ham_converter(fake_armonk_backend)

# Instantiate grape optimizer with this hamiltonian
grape_optimizer = QutipOptimizer(hamiltonian, fake_armonk_backend.configuration().dt)

# Create new QOCInstructionScheduleMap with this optimizer
grape_inst_map = QOCInstructionScheduleMap(grape_optimizer)

In [ ]:
# add measurement map to grape_inst_map
# this isn't long term sol, for measurement probably just use getter or something in instantiation
grape_inst_map._map['measure'] = fake_armonk_backend.defaults().instruction_schedule_map._map['measure']
# grape_inst_map._map['MEAS'] = fake_armonk_backend.defaults().instruction_schedule_map._map['MEAS']
if def_x:
    grape_inst_map._map['x'] = fake_armonk_backend.defaults().instruction_schedule_map._map['x']
grape_inst_map._qubit_instructions = fake_armonk_backend.defaults().instruction_schedule_map._qubit_instructions

In [ ]:
# create a random gate for testing
rand_gate = UnitaryGate(qiskit.quantum_info.random_unitary(2,2))
rand_gate.to_matrix()

In [ ]:
# Create circuit
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ = qiskit.QuantumCircuit(q, c)

# Add the X gate
# circ.x(q)
# circ.h(q)
circ.append(rand_gate, q)

# Add the measurement pulse
circ.measure([0], [0])

In [ ]:
# set the drive strength
omegad0 = 31919806.545849085
getattr(fake_armonk_backend.configuration(), 'hamiltonian')['vars']['omegad0'] = omegad0

# set the qubit frequency
freq_est = 4.97445401836326e9
fake_armonk_backend.configuration().qubit_freq_est=[freq_est]
getattr(fake_armonk_backend.configuration(), 'hamiltonian')['vars']['wq0'] = 2*np.pi*freq_est

In [ ]:
# generate model from backend
armonk_model = PulseSystemModel.from_backend(fake_armonk_backend)

In [ ]:
# construct the schedule from the circuit using the grape instruction_schedule_map
grape_schedule = qiskit.schedule(circ, inst_map = grape_inst_map,
                        meas_map = fake_armonk_backend.configuration().meas_map)

In [ ]:
# assemble qobj for job submission
backend_sim = PulseSimulator(configuration=fake_armonk_backend.configuration)

grape_qobj = assemble(grape_schedule,
                      backend=backend_sim,
                      qubit_lo_freq=[freq_est],
                      meas_level=2,
                      meas_return='single',
                      shots=1024)

In [ ]:
sim_result = backend_sim.run(grape_qobj, armonk_model).result()
sim_result.get_counts()

In [ ]:
noise_model = NoiseModel()
noise_model.add_basis_gates(['unitary'])
ideal_job = qiskit.execute(circ, QasmSimulator(),
                           basis_gates=noise_model.basis_gates)
ideal_job.result().get_counts()

In [ ]:
# now we run it on the real armonk
armonk_qobj = assemble(grape_schedule,
                      backend=real_armonk_backend,
                      qubit_lo_freq=[freq_est],
                      meas_level=2,
                      meas_return='single',
                      shots=1024)
real_job = real_armonk_backend.run(armonk_qobj)
job_id = real_job._job_id

In [ ]:
job = real_armonk_backend.retrieve_job(job_id)
job_monitor(job)
job.result().get_counts()

In [ ]:
print('simulated results: {}'.format(sim_result.get_counts()))
print('ideal results: {}'.format(ideal_job.result().get_counts()))
print('real results: {}'.format(real_job.result().get_counts()))
print('Grape run on {} backend'.format(which_armonk))
print('Used grape gate: {}'.format(not def_x))

In [ ]:
# def x gate on real armonk: 0:67, 1:957
# grape x gate on real armonk: 0:212, 1:812
# see https://qiskit.org/documentation/locale/de_DE/tutorials/pulse/5_pulse_simulator_backend_model.html